In [2]:
import pandas as pd
import plotly.express as px
try:
    counties_url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
    states_url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv'
    county_data = pd.read_csv(counties_url,dtype={"fips": str})
    state_data = pd.read_csv(states_url,dtype={"fips": str})
    print("Received latest NYT Data")
except:
    county_data = pd.read_csv('data/data/us-counties.csv',dtype={"fips": str})
    state_data = pd.read_csv('data/data/us-states.csv',dtype={"fips": str})
    print("Could not get latest NYT Data, falling back to local 4-3-20 data")
county_data = county_data.dropna()
state_data = state_data.dropna()

Received latest NYT Data


In [4]:
latest = county_data.query("date=='2020-04-15'")

In [55]:
total = state_data.groupby(state_data.date).sum()

In [18]:
total.head()

,cases,deaths
date,,
2020-01-21,1,0
2020-01-22,1,0
2020-01-23,1,0
2020-01-24,2,0
2020-01-25,3,0


In [19]:
fig = px.scatter(x=total.index, y=total.cases)
fig.show()

In [20]:
fig = px.scatter(x=total.index, y=total.deaths)
fig.show()

In [50]:

import plotly.graph_objects as go
fig_combo = go.Figure()

# Add traces
fig_combo.add_trace(go.Scatter(x=total.index, y=total.cases,
                    mode='lines+markers',
                               name='Cases USA',
                         marker=dict(size=2, color='#d7191c',
                                            line=dict(width=.5, color='#e36209')),
                                text=total.index,
                                hovertext=['Total Cases<br>{:,d}<br>'.format(
                                    i) for i in total.cases],
                                hovertemplate='<b>%{text}</b><br></br>' +
                                              '%{hovertext}' +
                                              '<extra></extra>'
                        
                        
                        
                        ))

fig_combo.add_trace(go.Scatter(x=total.index, y=total.deaths,
                    mode='lines+markers',
                               name = 'Deaths USA',
                         marker=dict(size=2, color='#626262',
                                            line=dict(width=.5, color='#626262')),
                                text=total.index,
                                hovertext=['Total Deaths<br>{:,d}<br>'.format(
                                    i) for i in total.deaths],
                                hovertemplate='<b>%{text}</b><br></br>' +
                                              '%{hovertext}' +
                                              '<extra></extra>'
                              ))
fig_combo.update_layout(
    margin=go.layout.Margin(
        l=10,
        r=10,
        b=10,
        t=5,
        pad=0
    ),
    yaxis=dict(
        showline=False, linecolor='#272e3e',
        zeroline=False,
        # showgrid=False,
        gridcolor='rgba(203, 210, 211,.3)',
        gridwidth=.1,
        #tickmode='array',
        # Set tick range based on the maximum number
        #tickvals=tickList,
        # Set tick label accordingly
        #ticktext=["{:.0f}k".format(i/1000) for i in tickList]
    ),
#    yaxis_title="Total Confirmed Case Number",
    xaxis=dict(
        showline=False, linecolor='#272e3e',
        showgrid=False,
        gridcolor='rgba(203, 210, 211,.3)',
        gridwidth=.1,
        zeroline=False
    ),
    xaxis_tickformat='%b %d',
#     hovermode='x',
    legend_orientation="h",
    # legend=dict(x=.02, y=.95, bgcolor="rgba(0,0,0,0)",),
    plot_bgcolor='#ffffff',
    paper_bgcolor='#ffffff',
    font=dict(color='#292929', size=10)
)


fig_combo.show()

In [44]:

import plotly.graph_objects as go
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=total.index, y=total.deaths,
                    mode='lines+markers',
                         marker=dict(size=2, color='#626262',
                                            line=dict(width=.5, color='#626262')),
                                text=total.index,
                                hovertext=['Total Deaths<br>{:,d}<br>'.format(
                                    i) for i in total.deaths],
                                hovertemplate='<b>%{text}</b><br></br>' +
                                              '%{hovertext}' +
                                              '<extra></extra>'
                        
                        
                        
                        ))

fig.update_layout(
    margin=go.layout.Margin(
        l=10,
        r=10,
        b=10,
        t=5,
        pad=0
    ),
    yaxis=dict(
        showline=False, linecolor='#272e3e',
        zeroline=False,
        # showgrid=False,
        gridcolor='rgba(203, 210, 211,.3)',
        gridwidth=.1,
        #tickmode='array',
        # Set tick range based on the maximum number
        #tickvals=tickList,
        # Set tick label accordingly
        #ticktext=["{:.0f}k".format(i/1000) for i in tickList]
    ),
#    yaxis_title="Total Confirmed Case Number",
    xaxis=dict(
        showline=False, linecolor='#272e3e',
        showgrid=False,
        gridcolor='rgba(203, 210, 211,.3)',
        gridwidth=.1,
        zeroline=False
    ),
    xaxis_tickformat='%b %d',
#     hovermode='x',
    legend_orientation="h",
    # legend=dict(x=.02, y=.95, bgcolor="rgba(0,0,0,0)",),
    plot_bgcolor='#ffffff',
    paper_bgcolor='#ffffff',
    font=dict(color='#292929', size=10)
)


fig.show()

In [87]:
difference = total.diff()
difference = difference.fillna(0)
difference.columns = ['case_increase','death_increase']
difference.head()

,case_increase,death_increase
date,,
2020-01-21,0.0,0.0
2020-01-22,0.0,0.0
2020-01-23,0.0,0.0
2020-01-24,1.0,0.0
2020-01-25,1.0,0.0


In [88]:
new = total.merge(difference, left_index=True, right_index=True)

In [91]:
new.head()

,cases,deaths,case_increase,death_increase
date,,,,
2020-01-21,1,0,0.0,0.0
2020-01-22,1,0,0.0,0.0
2020-01-23,1,0,0.0,0.0
2020-01-24,2,0,1.0,0.0
2020-01-25,3,0,1.0,0.0


In [95]:
new['case_percent'] = new.apply(lambda x: x.case_increase / (x.cases-x.case_increase),axis=1)
new['death_percent'] = new.apply(lambda x: x.death_increase / (x.deaths-x.death_increase),axis=1)

/Users/matthewhwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning:

invalid value encountered in double_scalars

/Users/matthewhwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning:

divide by zero encountered in double_scalars



In [97]:
new = new.fillna(0)

In [98]:
new

,cases,deaths,case_increase,death_increase,case_percent,death_percent
date,,,,,,
2020-01-21,1,0,0.0,0.0,0.000000,0.000000
2020-01-22,1,0,0.0,0.0,0.000000,0.000000
2020-01-23,1,0,0.0,0.0,0.000000,0.000000
2020-01-24,2,0,1.0,0.0,1.000000,0.000000
2020-01-25,3,0,1.0,0.0,0.500000,0.000000
2020-01-26,5,0,2.0,0.0,0.666667,0.000000
2020-01-27,5,0,0.0,0.0,0.000000,0.000000
2020-01-28,5,0,0.0,0.0,0.000000,0.000000
2020-01-29,5,0,0.0,0.0,0.000000,0.000000


In [105]:

import plotly.graph_objects as go
fig_daily = go.Figure()

# Add traces
fig_daily.add_trace(go.Scatter(x=new.index, y=new.case_increase,
                    mode='lines+markers',
                               name='Cases USA',
                         marker=dict(size=2, color='#d7191c',
                                            line=dict(width=.5, color='#e36209')),
                                text=total.index,
                                hovertext=['New Cases<br>{:,d}<br>'.format(
                                    int(i)) for i in new.case_increase],
                                hovertemplate='<b>%{text}</b><br></br>' +
                                              '%{hovertext}' +
                                              '<extra></extra>'
                        
                        
                        
                        ))

fig_daily.add_trace(go.Scatter(x=new.index, y=new.death_increase,
                    mode='lines+markers',
                               name = 'Deaths USA',
                         marker=dict(size=2, color='#626262',
                                            line=dict(width=.5, color='#626262')),
                                text=total.index,
                                hovertext=['New Deaths<br>{:,d}<br>'.format(
                                    int(i)) for i in new.death_increase],
                                hovertemplate='<b>%{text}</b><br></br>' +
                                              '%{hovertext}' +
                                              '<extra></extra>'
                              ))
fig_daily.update_layout(
    margin=go.layout.Margin(
        l=10,
        r=10,
        b=10,
        t=5,
        pad=0
    ),
    yaxis=dict(
        showline=False, linecolor='#272e3e',
        zeroline=False,
        # showgrid=False,
        gridcolor='rgba(203, 210, 211,.3)',
        gridwidth=.1,
        #tickmode='array',
        # Set tick range based on the maximum number
        #tickvals=tickList,
        # Set tick label accordingly
        #ticktext=["{:.0f}k".format(i/1000) for i in tickList]
    ),
#    yaxis_title="Total Confirmed Case Number",
    xaxis=dict(
        showline=False, linecolor='#272e3e',
        showgrid=False,
        gridcolor='rgba(203, 210, 211,.3)',
        gridwidth=.1,
        zeroline=False
    ),
    xaxis_tickformat='%b %d',
#     hovermode='x',
    legend_orientation="h",
    # legend=dict(x=.02, y=.95, bgcolor="rgba(0,0,0,0)",),
    plot_bgcolor='#ffffff',
    paper_bgcolor='#ffffff',
    font=dict(color='#292929', size=10)
)


fig_daily.show()